# FFXIV Event Templating Engine
This Jupyter Notebook creates message templates for FFXIV events.\
Each section guides you through the steps, what you need to do to create your own template.\
By default this Notebook comes with a ready made template for the 7th Haven community.\
You can create as many templates as you like, they will all be rendered and you can then just copy paste them to your\
servers and partake as soon as their markdown support releases.

## Setting up
The first 2 sections are just setup to make this Notebook work and create new functionality for the templating engine.\
These functions should not concern you, unless you're experience with Python and Jinja2 and want to create your own template extensions.

In [1]:
%pip install jinja2 pyyaml


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import jinja2
import datetime
def datetime_filter(value, format='%Y-%m-%d %H:%M%z'):
    dt = datetime.datetime.strptime(value, format)
    unixtime = int(dt.timestamp())
    return unixtime

def add_time(value, duration):
    return value + duration * 60

def sevenh_rp_level(value):
    levels = ["Low", "Medium Low", "Medium", "Medium High", "High"]
    return f"{value} - {levels[value-1]}"

j2_env = jinja2.Environment()
j2_env.filters["hammertime"] = datetime_filter
j2_env.filters["add_time"] = add_time
j2_env.filters["sevenh_rp_level"] = sevenh_rp_level


## Define Templates
Create all templates here that need to be rendered.

E.g.

[
    """\
    # {{ title }}\
    """,\
    """\
    \**{{ title }}**\
    """\
]

will render two templates, one with title as level 1 headline in markdown, and one with the title as bold text.


This notebook comes with template filter extensions for easier handling of dates.

**hammertime** converts a date from the format 2024-12-31 20:00 to a plain number (used in discord timestamps).

This number can then be used to format it in discord e.g. <t:13123:t> just the hours or <t:123123:f> for the full date time.

**add_time** will add a number of minutes to whatever hammertime has been created. This comes handy for reusing date times.\
e.g. {{ somedate | hammertime | add_time(60) }} will add 1 hour to whatever has been somedate originally.

In [3]:
import jinja2
templates = [
"""
# {{ event_title }}
{{ event_description }}

## Location
{{ location.server }} - {{ location.area }} - W{{ location.ward }} - P{{ location.plot }}

## Date and Time
**Doors open at:** <t:{{ date_time | hammertime }}:f>

## DJ Lineup
{%- for dj in djs %}
<t:{{ date_time | hammertime | add_time(dj.time_offset) }}:t> [{{ dj.name }}](<https://twitch.tv/{{ dj.twitch_handle }}>)
{%- endfor %}

## Further Links
:globe_with_meridians: [Website](<{{ links.website }}>)
:discord: [Discord](<https://discord.com/invite/{{ links.discord }}/>)
:instagram: [Instagram](<https://instagram.com/{{ links.instagram }}/>)
:partake: [Partake](<https://partake.gg/events/{{ links.partake }}/>)
""",
"""
{{ event_title }}
{{ event_description }}

Location
{{ location.server }} - {{ location.area }} - W{{ location.ward }} - P{{ location.plot }}

Date and Time
Doors open at: {{ date_time }}

DJ Lineup
{%- for dj in djs %}
https://twitch.tv/{{ dj.twitch_handle }}
{%- endfor %}

Further Links
{{ links.website }}
https://discord.com/invite/{{ links.discord }}
https://instagram.com/{{ links.instagram }}
"""
]

templates = map(j2_env.from_string, templates)

# Fill the Templates
Fill all the information below to automatically generate texts defined by your templates designed above.

In [4]:
import yaml

yml_string = """
event_title: Femme Fatale
venue_name: The Black Wolf
location:
  server: Odin
  area: Mist
  ward: 29
  plot: 2
date_time: 2024-03-13 20:00+0100
nsfw: Yes
rp_level: 3
links:
  website: https://theblackwolf.rocks
  discord: Z2cRA3ygh7
  instagram: theblackwolf.xiv
  partake: 36108
djs:
  - name: Astria Fae
    time_offset: 0
    twitch_handle: AstriaFae
  - name: Blue Mantra
    time_offset: 120
    twitch_handle: blue_mantra
event_description: |
  The international women's day has just ended, but not with us!
  We continue to celebrate at our venue with our special event "Femme Fatale".

  And right on time our resident DJ Blue Mantra came back to be part of this.
  So give her a warm hug and welcome her back with us. She will be accompanied by the lovely 
  Astria Fae who will be opening the event for us.
  These maidens of metal will make sure to keep you busy on the floor and in the pits.

  As usual we will have a well crafted special drink for the event and we will be waiting for you to join us.
  Hit up our staff at the bar and they will be happy to serve you.


  We are looking forward to seeing you all there and have a great time together.
"""

data = yaml.safe_load(yml_string)

for template in templates:
    print(template.render(data))
    print("============")



__**Femme Fatale**__

__**Venue/Event Name**__: The Black Wolf
__**Server**__: Odin
__**Location**__: Mist W29 P2
__**Date and Time**__: <t:1710356400:f>
__**NSFW/Mature Venue/Event?**__: True
__**Level of RP immersion**__: 3 - Medium
__**Event Description:**__ 
The international women's day has just ended, but not with us!
We continue to celebrate at our venue with our special event "Femme Fatale".

And right on time our resident DJ Blue Mantra came back to be part of this.
So give her a warm hug and welcome her back with us. She will be accompanied by the lovely 
Astria Fae who will be opening the event for us.
These maidens of metal will make sure to keep you busy on the floor and in the pits.

As usual we will have a well crafted special drink for the event and we will be waiting for you to join us.
Hit up our staff at the bar and they will be happy to serve you.


We are looking forward to seeing you all there and have a great time together.


__**DJ's / Music**__
<t:1710356400:t